In [1]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np

parent_dir = os.path.dirname(os.getcwd())
print(parent_dir)

/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-7-24


'/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-7-24/Data Embedding'

In [2]:
loaded_model = tf.keras.models.load_model(
os.path.join(parent_dir, 'Data Embedding & Model Development/Classification Models/Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Fold 2.keras'))

In [4]:
#loading triple embeddings from robokops kg
df = pd.read_csv(os.path.join(parent_dir,'Data Embedding & Model Development/Data Mined Embedded Dataset/ROBOKOP Text Mined Embedded Dataset.csv')
)
df = df.drop('Unnamed: 0', axis = 1)
df = df.drop_duplicates(subset = ['0','2','4'])


In [5]:
print(df)

                 0            1                                     2  \
0       vandetanib  CHEBI:49960  multiple endocrine neoplasia type 2a   
1        ponatinib  CHEBI:78543  multiple endocrine neoplasia type 2a   
2        sorafenib  CHEBI:50924  multiple endocrine neoplasia type 2a   
3      regorafenib  CHEBI:68647  multiple endocrine neoplasia type 2a   
4      ruxolitinib  CHEBI:66919  multiple endocrine neoplasia type 2a   
...            ...          ...                                   ...   
89475  midostaurin  CHEBI:63452                          liver cancer   
89478     axitinib  CHEBI:66910                          liver cancer   
89479   canertinib  CHEBI:61399                          liver cancer   
89482    diazoxide   CHEBI:4495                          liver cancer   
89483  ceftriaxone  CHEBI:29007                          liver cancer   

                   3                                             4       5  \
0       DOID:0050430                         

In [9]:
total_df = pd.concat([model_df,df],axis=0)
columns = ['0','1','2']
unique = total_df.drop_duplicates(subset= columns, keep = 'first')
new_df = unique.iloc[17414:]
new_df = new_df.reset_index().drop('index', axis=1)
new_df = new_df.drop_duplicates(subset= columns)
print(new_df)

                 0                                     1  \
0        ponatinib  multiple endocrine neoplasia type 2a   
1        sorafenib  multiple endocrine neoplasia type 2a   
2      regorafenib  multiple endocrine neoplasia type 2a   
3      ruxolitinib  multiple endocrine neoplasia type 2a   
4        sunitinib  multiple endocrine neoplasia type 2a   
...            ...                                   ...   
45943   canertinib                          liver cancer   
45944    diazoxide                          liver cancer   
45945  ceftriaxone                          liver cancer   
45946       pi-103                          liver cancer   
45947       bkm120                          liver cancer   

                                                       2         3         4  \
0                                     ret proto-oncogene -0.430430  0.251120   
1                                     ret proto-oncogene  0.741410  0.247970   
2                                     r

In [6]:
data = df.iloc[:, 6:-1]

print(data[:1])


         6        7        8        9       10       11        12       13  \
0  0.79093  0.60086 -0.28079  0.40022 -0.50199  0.65795  0.013582 -0.07678   

         14       15  ...      596     597      598      599       600  \
0 -0.020119 -0.31451  ... -0.27445 -0.5197 -0.23558  0.54809  0.650776   

        601       602      603      604      605  
0 -0.738723  0.183145 -0.23919  0.72017 -0.22782  

[1 rows x 600 columns]


In [7]:
predictions = loaded_model.predict(data)
data_rounded = np.round(predictions)
descending_confidence_values = np.sort(predictions.flatten())[::-1]
ascending_indices = np.argsort(predictions.flatten())
descending_indices = np.flip(ascending_indices)

1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 790us/step


In [8]:
print(ascending_indices[:5])

[16838 16840 27899 44529 16262]


In [9]:
print(predictions[1920])

[0.93079]


In [10]:
print(df.iloc[1920])

0                               roxindole
1                             CHEBI:48558
2      postencephalitic parkinson disease
3                           MONDO:0001945
4                    dopamine receptor d1
                      ...                
602                              0.132967
603                              0.634726
604                               0.86164
605                             -1.299339
606                                     1
Name: 4100, Length: 607, dtype: object


In [11]:
## finding the indices of only the 1's (actives)

true_positive_indices = []
true_positive_count = 0

false_negative_indices = []
false_negative_count = 0

for i in range (0,len(data_rounded)):
    if data_rounded[i] == 1:
        true_positive_indices.append(i)
        true_positive_count += 1
    else:
        false_negative_indices.append(i)
        false_negative_count +=1

print(true_positive_indices[:20])
print(true_positive_count)
print(false_negative_indices[:20])
print(false_negative_count)

[0, 1, 2, 3, 4, 5, 7, 9, 10, 16, 20, 21, 22, 23, 51, 52, 53, 87, 88, 89]
21374
[6, 8, 11, 12, 13, 14, 15, 17, 18, 19, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
26024


In [12]:
print(df.iloc[1,:6])

0                               ponatinib
1                             CHEBI:78543
2    multiple endocrine neoplasia type 2a
3                            DOID:0050430
4                      ret proto-oncogene
5                                  P07949
Name: 1, dtype: object


In [13]:
prediction_array = []
for i in descending_indices:
    triple_name = df.iloc[i,:6].tolist()
    conf_value = float(predictions[i])
    row = [triple_name,conf_value]
    flat_row = row[0]+[row[1]]
    prediction_array.append(flat_row)



/var/folders/91/rbzkmg7n5g7f992d83ryjnz40000gn/T/ipykernel_54158/2981000910.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  conf_value = float(predictions[i])


In [14]:
print(prediction_array[:2])

[['promazine', 'CHEBI:8459', 'major depressive disorder', 'MONDO:0002009', 'histamine receptor h1', 'P35367', 0.9999656081199646], ['hydroxyzine', 'CHEBI:5818', 'major depressive disorder', 'MONDO:0002009', 'histamine receptor h1', 'P35367', 0.9999567866325378]]


In [25]:
to_send = prediction_array[:30]

In [15]:
columns_to_use = ['drug_name','drug_id','disease_name','disease_id','protein_name','protein_id','model_confidence_value']
df = pd.DataFrame(data = prediction_array,columns = columns_to_use)
df = df.drop_duplicates(subset= ['drug_name','disease_name','protein_name'])
df.to_csv(os.path.join(os.getcwd(),'Screening Results/Complete Set 1 Top Model Predictions of Top ROBOKOP Data Mined Triples.csv'))

In [26]:
print(df)

             drug_name       drug_id               disease_name  \
0            promazine    CHEBI:8459  major depressive disorder   
2          hydroxyzine    CHEBI:5818  major depressive disorder   
4      brompheniramine    CHEBI:3183  major depressive disorder   
6            bromazine   CHEBI:59177  major depressive disorder   
7             loxapine   CHEBI:50841        depressive disorder   
...                ...           ...                        ...   
89580     dexibuprofen   CHEBI:43415                     ptosis   
89581     pimavanserin  CHEBI:133017        gingival overgrowth   
89582     dexibuprofen   CHEBI:43415                  exotropia   
89583      risperidone    CHEBI:8871               cornea plana   
89584     pimavanserin  CHEBI:133017               cornea plana   

          disease_id                   protein_name protein_id  \
0      MONDO:0002009          histamine receptor h1     P35367   
2      MONDO:0002009          histamine receptor h1     P35367 